<a href="https://colab.research.google.com/github/MMRES-PyBootcamp/MMRES-python-bootcamp2022/blob/main/08_scipy_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 8 - Statistics in Python (Second part)

sources:

https://medium.com/insights-school/learn-basic-statistics-with-python-cc0f45275929

https://scipy-lectures.org/intro/scipy.html#scipy


In [ ]:
import math
import statistics
from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv('https://scipy-lectures.org/_downloads/brain_size.csv', sep=';', na_values=".")
data

## Pearson Correlation
We can  compute Pearson correlation coefficient using SciPy’s stats module. SciPy’s stats module has a function called pearsonr() that can take two arrays and return a tuple containing Pearson correlation coefficient and the significance of the correlation as p-value.


In [ ]:
#The first element of tuple is the Pearson correlation and the second is p-value.
stats.pearsonr(data['FSIQ'],data['VIQ'])


In [ ]:
# we can also use pandas
data['FSIQ'].corr(data['VIQ'], method="pearson")


## Spearman Correlation
Pearson correlation assumes that the data we are comparing is normally distributed. When that assumption is not true, the correlation value is not reflecting the true association. Spearman correlation does not assume that data is from a specific distribution, so it is a non-parametric correlation measure. Spearman correlation is also known as Spearman’s rank correlation as it computes correlation coefficient on rank values of the data. Using SciPy, we can compute Spearman correlation using the function spearmanr().

In [ ]:
#The first element of tuple is the Spearman's rank correlation and the second is p-value.
stats.spearmanr(data['FSIQ'],data['VIQ'])

In [ ]:
# we can also use pandas
data['FSIQ'].corr(data['VIQ'], method="spearman")


In [ ]:
slope, intercept, r_value, pv, se = stats.linregress(data['FSIQ'],data['VIQ'])
sns.regplot(x="FSIQ", y="VIQ", data=data, 
      ci=None, label="y={0:.2f}x+{1:.2f}".format(slope, intercept)).legend(loc="best")

## Student’s t-test: the simplest statistical test
### 1-sample t-test: testing the value of a population mean
scipy.stats.ttest_1samp() tests if the population mean of data is likely to be equal to a given value (technically if observations are drawn from a Gaussian distributions of given population mean). It returns the T statistic, and the p-value:

In [ ]:
sns.distplot(data['VIQ'],hist=False)
plt.axvline(0)
res=stats.ttest_1samp(data['VIQ'], 0) 
print (res[0])



## 2-sample t-test: testing for difference across populations
We have seen above that the mean VIQ in the male and female populations were different. To test if this is significant, we do a 2-sample t-test with scipy.stats.ttest_ind():

In [ ]:

female_viq = data[data['Gender'] == 'Female']['VIQ']
male_viq = data[data['Gender'] == 'Male']['VIQ']
stats.ttest_ind(female_viq, male_viq) 



In [ ]:
sns.distplot(female_viq,hist=False,color='purple')
sns.distplot(male_viq,hist=False,color='blue')


## Paired tests: repeated measurements on the same individuals
PIQ, VIQ, and FSIQ give 3 measures of IQ. Let us test if FISQ and PIQ are significantly different. We can use a 2 sample test:

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])
sns.distplot(data['FSIQ'],color='r')
sns.distplot(data['PIQ'],color='g')

# try to use groupby to run a two-sample t-test

The problem with this approach is that it forgets that there are links between observations: FSIQ and PIQ are measured on the same individuals. Thus the variance due to inter-subject variability is confounding, and can be removed, using a “paired test”, or “repeated measures test”:

In [ ]:
stats.ttest_rel(data['FSIQ'], data['PIQ']) 

This is equivalent to a 1-sample test on the difference:

In [ ]:
sns.distplot(data['FSIQ'] - data['PIQ'])
plt.axvline(0)
stats.ttest_1samp(data['FSIQ'] - data['PIQ'], 0) 


T-tests assume Gaussian errors. We can use a Wilcoxon signed-rank test, that relaxes this assumption. Note The corresponding test in the non paired case is the Mann–Whitney U test, scipy.stats.mannwhitneyu().

In [ ]:
stats.wilcoxon(data['FSIQ'], data['PIQ'])   

### Exercise

Test the difference between weights in males and females.
Use non parametric statistics to test the difference between VIQ in males and females.
Conclusion: we find that the data does not support the hypothesis that males and females have different VIQ.

# Linear models, multiple factors, and analysis of variance
Given two set of observations, x and y, we want to test the hypothesis that y is a linear function of x. In other terms:

> y = ax + b + e

where e is observation noise. We will use the statsmodels module to:

Fit a linear model. We will use the simplest strategy, ordinary least squares (OLS).
Test that coef is non zero.

In [ ]:
x = np.linspace(-5, 5, 20)
x2= np.linspace(0,10, 20)
x3=  np.random.normal(size=20)

np.random.seed(1)
# normal distributed noise
y = -5 + 3*x -10*x2 + 4 * np.random.normal(size=x.shape)
# Create a data frame containing all the relevant variables
data = pd.DataFrame({'x': x,'x2': x2, 'x3':x3, 'y': y})

In [ ]:
data

In [ ]:
from statsmodels.formula.api import ols
model = ols("y ~ x + x2 + x3", data).fit()
print(model.summary())

## Categorical variables:
 comparing groups or multiple categories


In [ ]:
data = pd.read_csv('https://scipy-lectures.org/_downloads/brain_size.csv', sep=';', na_values=".")
model = ols("VIQ ~ Gender + 1", data).fit()
print(model.summary())  


## Link to t-tests between different FSIQ and PIQ

To compare different types of IQ, we need to create a “long-form” table, listing IQs, where the type of IQ is indicated by a categorical variable:

In [ ]:
data_fisq = pd.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pd.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pd.concat((data_fisq, data_piq))
print(data_long)

#sns.boxplot(data=data_long,x='type',y='iq')
sns.violinplot(data=data_long,x='type',y='iq')



In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())


## Multiple Regression: 
including multiple factors

In [ ]:
data = pd.read_csv('https://scipy-lectures.org/_downloads/iris.csv')
model = ols('sepal_width ~ name + petal_length', data).fit()
print(model.summary())  

# Analysis of variance (ANOVA)
In the above iris example, we wish to test if the petal length is different between versicolor and virginica, after removing the effect of sepal width. This can be formulated as testing the difference between the coefficient associated to versicolor and virginica in the linear model estimated above (it is an Analysis of Variance, ANOVA). For this, we write a vector of ‘contrast’ on the parameters estimated: we want to test "name[T.versicolor] - name[T.virginica]", with an F-test.

In [ ]:
print(model.f_test([0, 1, -1, 0]))  